<a href="https://colab.research.google.com/github/Makorg123/Python-EDA/blob/main/MLUsingPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import matplotlib.pyplot as plt
import pandas as pd
import pyspark

In [4]:
# Create a Spark Session

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Regression").getOrCreate()

In [5]:
spark

In [6]:
salesdf = spark.read.csv('/content/drive/MyDrive/train.csv',inferSchema=True,header=True)

In [7]:
# Top 5 records
salesdf.show(5)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
|1000001| P00087842|     F|0-17|        10|            A|                         2|             0|                12|              null|              null|    1422|
|100

In [8]:
from pyspark.sql.functions import isnull,when,count,col

In [9]:
# To find the missing values.
salesdf.select([count(when(isnull(c),c)).alias(c)
for c in salesdf.columns]).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|            173638|            383247|       0|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



In [11]:
salesdf.groupby('Product_Category_2').count().show()

+------------------+------+
|Product_Category_2| count|
+------------------+------+
|                12|  5528|
|              null|173638|
|                13| 10531|
|                 6| 16466|
|                16| 43255|
|                 3|  2884|
|                 5| 26235|
|                15| 37855|
|                 9|  5693|
|                17| 13320|
|                 4| 25677|
|                 8| 64088|
|                 7|   626|
|                10|  3043|
|                11| 14134|
|                14| 55108|
|                 2| 49217|
|                18|  2770|
+------------------+------+



In [12]:
# Impute with code 999
salesdf = salesdf.na.fill(value=999,subset=['Product_Category_2'])

In [13]:
salesdf.groupby('Product_Category_2').count().show()

+------------------+------+
|Product_Category_2| count|
+------------------+------+
|                12|  5528|
|                13| 10531|
|                 6| 16466|
|                16| 43255|
|                 3|  2884|
|                 5| 26235|
|                15| 37855|
|                 9|  5693|
|                17| 13320|
|                 4| 25677|
|                 8| 64088|
|                 7|   626|
|                10|  3043|
|                11| 14134|
|                14| 55108|
|                 2| 49217|
|               999|173638|
|                18|  2770|
+------------------+------+



In [14]:
# Check null values for Product_Category_3
salesdf.groupby('Product_Category_3').count().show()

+------------------+------+
|Product_Category_3| count|
+------------------+------+
|                12|  9246|
|              null|383247|
|                13|  5459|
|                16| 32636|
|                 6|  4890|
|                 3|   613|
|                 5| 16658|
|                15| 28013|
|                17| 16702|
|                 9| 11579|
|                 4|  1875|
|                 8| 12562|
|                10|  1726|
|                11|  1805|
|                14| 18428|
|                18|  4629|
+------------------+------+



In [15]:
salesdf = salesdf.na.fill(value=999,subset=['Product_Category_3'])

In [16]:
salesdf.groupby('Product_Category_3').count().show()

+------------------+------+
|Product_Category_3| count|
+------------------+------+
|                12|  9246|
|                13|  5459|
|                16| 32636|
|                 6|  4890|
|                 3|   613|
|                 5| 16658|
|                15| 28013|
|                17| 16702|
|                 9| 11579|
|                 4|  1875|
|                 8| 12562|
|                10|  1726|
|                11|  1805|
|                14| 18428|
|               999|383247|
|                18|  4629|
+------------------+------+



In [17]:
salesdf.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [18]:
# How many Male & female are there?
salesdf.groupby('Gender').count().show()

+------+------+
|Gender| count|
+------+------+
|     F|135809|
|     M|414259|
+------+------+



In [21]:
# Average Purchase by Gender
mean_purchase = salesdf.groupby('Gender').mean('Purchase').show()

+------+-----------------+
|Gender|    avg(Purchase)|
+------+-----------------+
|     F|8734.565765155476|
|     M|9437.526040472265|
+------+-----------------+



In [24]:
from pyspark.sql.functions import round

In [29]:
mean_purchase.select('Gender', round('avg(Purchase)', 2).alias('Avg_Purchase'))

AttributeError: ignored

In [20]:
# Average Purchase by Age
salesdf.groupby('Age').mean('Purchase').show()

+-----+-----------------+
|  Age|    avg(Purchase)|
+-----+-----------------+
|18-25|9169.663606261289|
|26-35|9252.690632869888|
| 0-17|8933.464640444974|
|46-50|9208.625697468327|
|51-55|9534.808030960236|
|36-45|9331.350694917874|
|  55+|9336.280459449405|
+-----+-----------------+



In [27]:
salesdf.select('Purchase').describe().show()

+-------+-----------------+
|summary|         Purchase|
+-------+-----------------+
|  count|           550068|
|   mean|9263.968712959126|
| stddev|5023.065393820575|
|    min|               12|
|    max|            23961|
+-------+-----------------+



In [31]:
# Skwness and Kurtosis

from pyspark.sql.functions import skewness,kurtosis,median

In [32]:
salesdf.select(skewness('Purchase'),kurtosis('Purchase'),median('Purchase')).show()

+------------------+-------------------+----------------+
|skewness(Purchase)| kurtosis(Purchase)|median(Purchase)|
+------------------+-------------------+----------------+
|0.6001383671643392|-0.3383853975360327|          8047.0|
+------------------+-------------------+----------------+



In [34]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer # Like get_dummies and label encoder

In [36]:
indexer = [StringIndexer(inputCol = column, outputCol = column + 'index').fit(salesdf)
for column in list(set(salesdf.columns)-set(['Purchase','User_ID','Product_ID']))]

In [37]:
pipeline = Pipeline(stages=indexer)
salesdf  = pipeline.fit(salesdf).transform(salesdf)

In [38]:
salesdf.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase',
 'City_Categoryindex',
 'Product_Category_1index',
 'Stay_In_Current_City_Yearsindex',
 'Product_Category_2index',
 'Genderindex',
 'Marital_Statusindex',
 'Ageindex',
 'Occupationindex',
 'Product_Category_3index']

In [41]:
salesdf.show(15)

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+------------------+-----------------------+-------------------------------+-----------------------+-----------+-------------------+--------+---------------+-----------------------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|City_Categoryindex|Product_Category_1index|Stay_In_Current_City_Yearsindex|Product_Category_2index|Genderindex|Marital_Statusindex|Ageindex|Occupationindex|Product_Category_3index|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+------------------+-----------------------+-------------------------------+-----------------------+-----------+-------------------+--------+

In [42]:
columnstodrop = ['User_ID','Product_ID','Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years','Marital_Status',
                 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']

In [43]:
# Drop the columns
salesdf = salesdf.drop(*columnstodrop)

In [44]:
salesdf.show(10)

+--------+------------------+-----------------------+-------------------------------+-----------------------+-----------+-------------------+--------+---------------+-----------------------+
|Purchase|City_Categoryindex|Product_Category_1index|Stay_In_Current_City_Yearsindex|Product_Category_2index|Genderindex|Marital_Statusindex|Ageindex|Occupationindex|Product_Category_3index|
+--------+------------------+-----------------------+-------------------------------+-----------------------+-----------+-------------------+--------+---------------+-----------------------+
|    8370|               2.0|                    6.0|                            1.0|                    0.0|        1.0|                0.0|     6.0|           12.0|                    0.0|
|   15200|               2.0|                    1.0|                            1.0|                    8.0|        1.0|                0.0|     6.0|           12.0|                    3.0|
|    1422|               2.0|                

In [45]:
from pyspark.ml.feature import RFormula

In [47]:
RFormula,
formula = RFormula(formula='Purchase~.',featuresCol='features',labelCol='label')

In [48]:
salesdf = formula.fit(salesdf).transform(salesdf)

In [49]:
salesdf.select('features','label').show(5)

+--------------------+-------+
|            features|  label|
+--------------------+-------+
|[2.0,6.0,1.0,0.0,...| 8370.0|
|[2.0,1.0,1.0,8.0,...|15200.0|
|[2.0,12.0,1.0,0.0...| 1422.0|
|[2.0,12.0,1.0,2.0...| 1057.0|
|[1.0,2.0,3.0,0.0,...| 7969.0|
+--------------------+-------+
only showing top 5 rows



In [50]:
# Building the model
from pyspark.ml.regression import LinearRegression

In [51]:
reg = LinearRegression(featuresCol = 'features', labelCol = 'label')

In [52]:
regmodel = reg.fit(salesdf)

In [53]:
regmodel.summary.r2adj

0.06356300117930425

In [54]:
regmodel.summary.rootMeanSquaredError

4860.759869813398

In [55]:
from pyspark.ml.regression import DecisionTreeRegressor

In [65]:
tree = DecisionTreeRegressor(featuresCol = 'features',labelCol = 'label',maxDepth=8)

In [66]:
treemodel = tree.fit(salesdf)

In [67]:
treepredict = treemodel.transform(salesdf)

In [68]:
from pyspark.ml.evaluation import RegressionEvaluator

In [69]:
rsquare = RegressionEvaluator(predictionCol='prediction',labelCol='label',metricName='r2')

In [70]:
rmse = RegressionEvaluator(predictionCol = 'prediction', labelCol = 'label', metricName = 'rmse')

In [71]:
rsquare.evaluate(treepredict)

0.6552517597578954

In [72]:
rmse.evaluate(treepredict)

2949.303501228663